In [15]:
import numpy as np
import json

class ProblemSolver:
    def __init__(self, actions, epsilon=0.1, gamma=0.99, alpha=0.1, lambd=0.9):
        self.actions = actions
        self.epsilon = epsilon
        self.gamma = gamma
        self.alpha = alpha
        self.lambd = lambd
        self.Q = {}  # Q-values table
        self.e = {}  # Eligibility traces table

    def choose_action(self, sq):
        sq_tuple = tuple(sq)  # Convert numpy array to tuple
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.actions)
        else:
            if sq_tuple in self.Q:  # Use the tuple as the key
                return np.argmax(self.Q[sq_tuple])
            else:
                return np.random.choice(self.actions)


    def update_Q(self, sq, action, reward, next_sq, next_action):
        if sq not in self.Q:
            self.Q[sq] = np.zeros(self.actions)
            self.e[sq] = np.zeros(self.actions)

        delta = reward + self.gamma * self.Q.get(next_sq, np.zeros(self.actions))[next_action] - self.Q[sq][action]
        self.e[sq][action] += 1

        for state in self.Q:
            for a in range(self.actions):
                self.Q[state][a] += self.alpha * delta * self.e[state][a]
                self.e[state][a] *= self.gamma * self.lambd

    def solve_problem(self, sq, reward, next_sq):
        action = self.choose_action(sq)
        if reward is not None:  # If reward is received
            next_action = self.choose_action(next_sq)
            self.update_Q(sq, action, reward, next_sq, next_action)
            sq = next_sq
            return action, sq
        else:
            return action, sq

def retrieve(sq, C_B, threshold=0.5):
    sim_candidates = []

    for c in C_B:
        sim_value = sim_q(sq, c)
        if sim_value == 1:
            return c
        elif sim_value > threshold:
            sim_candidates.append((sim_value, c))

    if not sim_candidates:
        return None

    sim_candidates.sort(reverse=True)  # Sort in descending order of similarity value
    return sim_candidates[0][1]  # Return the case with the highest similarity value

def reuse(c, tc):
    hashed_c = tuple(array.tobytes() if isinstance(array, np.ndarray) else array for array in c)
    tc[hashed_c] = c

def revise(self, episode_ended_successfully):
    new_C_B = {}  # Create a new dictionary to store revised cases
    for key, value in self.C_B.items():
        if isinstance(key, np.ndarray):
            key_tuple = tuple(key.tolist())  # Convert numpy arrays to tuples
        elif not isinstance(key, tuple):
            key_tuple = (key,)  # Wrap non-tuple keys in a tuple
        else:
            key_tuple = key
        
        if episode_ended_successfully and key_tuple in self.C_B:
            new_C_B[key_tuple] = value + 1  # Increment trust value of the case
        elif not episode_ended_successfully and key_tuple in self.C_B:
            new_C_B[key_tuple] = value - 1  # Decrement trust value of the case
    self.C_B = new_C_B

def retain(C_B, episode_ended_successfully, threshold=0):
    for c in C_B:
        if episode_ended_successfully:  # Check if episode ended successfully
            # Convert keys to tuples if necessary
            key = tuple(array.tolist() if isinstance(array, np.ndarray) else array for array in c)
            if key not in C_B:
                C_B[key] = 1  # Insert c into C_B with initial trust value 1
            else:
                C_B[key] += 1  # Increment trust value of c in C_B

    # Remove cases from C_B with trust value less than threshold
    cases_to_remove = [key for key, trust_value in C_B.items() if trust_value < threshold]
    for key in cases_to_remove:
        del C_B[key]

def sim_q(sq, c):
    # Example of a similarity function comparing qualitative states
    similarity = np.random.rand()  # Replace this with your own similarity calculation
    return similarity

class QCBRL:
    def __init__(self, actions, threshold=0.5, epsilon=0.1, gamma=0.99, alpha=0.1, lambd=0.9):
        self.problem_solver = ProblemSolver(actions, epsilon, gamma, alpha, lambd)
        self.C_B = {}
        self.threshold = threshold

    def train(self, episodes, max_steps):
        for _ in range(episodes):
            state = self.reset_environment()
            for _ in range(max_steps):
                action, next_state = self.take_action(state)
                reward, next_state = self.observe_reward(next_state)
                c = (state, action, reward, next_state)
                self.reuse(c)
                # Determine if the episode ended successfully
                episode_ended_successfully = (reward is not None)
                if episode_ended_successfully:
                    new_C_B = {}
                    for key, value in self.C_B.items():
                        if isinstance(key, tuple):
                            new_key = key
                        else:
                            new_key = (key,)
                        new_C_B[new_key] = value
                    self.C_B = new_C_B
                state = next_state
                if reward is not None:
                    break

    def reset_environment(self):
        return np.random.rand(4)  # Example of resetting the environment, replace with your environment's reset function

    def take_action(self, state):
        action, next_state = self.problem_solver.solve_problem(state, None, None)
        return action, next_state

    def observe_reward(self, state):
        # Example of taking an action in the environment and getting the reward and next state
        next_state = state + np.random.randn(4)  # Random next state
        reward = np.random.randn()  # Random reward
        return reward, next_state

    def reuse(self, c):
        reuse(c, self.C_B)

    def revise(self, episode_ended_successfully):
        revise(self, episode_ended_successfully)

    def retain(self, episode_ended_successfully):
        retain(self.C_B, episode_ended_successfully, self.threshold)

    def convert_to_serializable(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            converted_dict = {}
            for k, v in obj.items():
                if isinstance(k, tuple):
                    k = '_'.join(map(str, k))  # Convert tuple keys to strings
                converted_dict[self.convert_to_serializable(k)] = self.convert_to_serializable(v)
            return converted_dict
        elif isinstance(obj, list):
            return [self.convert_to_serializable(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(self.convert_to_serializable(item) for item in obj)
        else:
            return obj

    def save_case_base(self, filename):
        # Convert NumPy arrays to lists and tuple keys to string keys
        converted_case_base = self.convert_to_serializable(self.C_B)
        with open(filename, 'w') as file:
            json.dump(converted_case_base, file)
    def load_case_base(self, filename):
        with open(filename, 'rb') as f:
            self.C_B = pickle.load(f)

# Example usage
if __name__ == "__main__":
    actions = 3  # Example number of actions
    agent = QCBRL(actions)
    agent.train(episodes=100, max_steps=100)
    agent.save_case_base("case_base.json")


TypeError: keys must be str, int, float, bool or None, not tuple